In [1]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv

from xcontest.xcontest import login, flight_days
from wetter.wetter import load_raw_dataframes,get_wind_data,get_rain_data

load_dotenv()

FLUGBAHN_DIR = 30 # 03 & 21

timeframe_start,timeframe_end = 2021010100,2022123123

df_wind = get_wind_data(timeframe_start,timeframe_end , 13)
df_rain = get_rain_data(timeframe_start,timeframe_end , 13)
df_wind = df_wind.join(df_rain.set_index("MESS_DATUM"), on="MESS_DATUM", validate="1:1" )


df_wind['wind_alignment'] = df_wind.apply(lambda row: np.sin((row['direction'] - FLUGBAHN_DIR )* np.pi / 180.)**2 , axis=1)

print(df_wind.head())

driver = login()

flight_day_list= flight_days(driver, 'Metzingen')

        MESS_DATUM  strength  direction  precip  wind_alignment
621941  2021010113       3.4        260     0.0        0.586824
621965  2021010213       1.4        230     0.0        0.116978
621989  2021010313       6.2         30     0.2        0.000000
622013  2021010413       4.2         50     0.3        0.116978
622037  2021010513       3.7         50     0.0        0.116978
['10.02.2024', '09.09.2023', '11.08.2023', '05.08.2023', '07.07.2023', '18.06.2023', '10.06.2023', '04.06.2023', '03.06.2023', '28.05.2023', '27.05.2023', '18.05.2023', '30.04.2023', '23.04.2023', '07.04.2023', '18.03.2023', '30.10.2022', '09.10.2022', '30.09.2022', '21.08.2022', '02.07.2022', '04.06.2022', '03.06.2022', '18.04.2022', '06.03.2022', '15.01.2022', '24.10.2021', '17.10.2021', '10.10.2021', '02.10.2021', '21.08.2021', '12.08.2021', '25.07.2021', '10.07.2021', '04.07.2021', '27.06.2021', '26.06.2021', '01.06.2021', '30.05.2021', '08.05.2021', '01.05.2021', '17.10.2020', '20.09.2020', '17.09.2020',

In [19]:
from datetime import datetime

flight_day_as_mess_date = [int(datetime.strftime(datetime.strptime(d,'%d.%m.%Y'), '%Y%m%d13')) for d in flight_day_list]

#print([d for d in flight_day_as_mess_date if d >= timeframe_start and d <= timeframe_end])

df_wind['is_flight_day'] = df_wind.apply(lambda row: 1 if row['MESS_DATUM'] in flight_day_as_mess_date else 0, axis=1)
df_wind['bias'] = df_wind.apply(lambda row: 1 , axis=1)

df_wind[df_wind['is_flight_day']==1].head()

X, y = df_wind[['bias','strength','wind_alignment','precip']].to_numpy(), df_wind['is_flight_day'].to_numpy() # features and labels
print(X.shape, y.shape)

# perceptron learn ?
# https://en.wikipedia.org/wiki/Perceptron
def softmax_stable(x):
    return(np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum())

# init 
r = 0.1
wt = np.asarray([[0.,0.,0.,0.]]) # X.shape[1] + 1

steps =0
while steps < 20:
    for j,x in enumerate(X):
        #print(x)
        yt = softmax_stable(wt.dot(x))
        #print(w.dot(x), yt)
        wt = wt + r *(y[j]-yt)*x 

    print(wt)
    steps += 1


(724, 4) (724,)
[[ -69.9        -328.82        -40.52348363  494.9       ]]
[[-139.8        -657.64        -81.04696726  989.8       ]]
[[-209.7        -986.46       -121.57045089 1484.7       ]]
[[ -279.6        -1315.28        -162.09393452  1979.6       ]]
[[ -349.5        -1644.1         -202.61741815  2474.5       ]]
[[ -419.4        -1972.92        -243.14090178  2969.4       ]]
[[ -489.3        -2301.74        -283.66438541  3464.3       ]]
[[ -559.2        -2630.56        -324.18786904  3959.2       ]]
[[ -629.1        -2959.38        -364.71135267  4454.1       ]]
[[ -699.        -3288.2        -405.2348363  4949.       ]]
[[ -768.9        -3617.02        -445.75831993  5443.9       ]]
[[ -838.8        -3945.84        -486.28180356  5938.8       ]]
[[ -908.7        -4274.66        -526.80528719  6433.7       ]]
[[ -978.6        -4603.48        -567.32877082  6928.6       ]]
[[-1048.5        -4932.3         -607.85225445  7423.5       ]]
[[-1118.4        -5261.12        -648.37